In [ ]:
#对序列进行embedding，并拼接
from gensim.models import Word2Vec
#选择要embedding的序列，可以分别为客户标签和还款记录
data = pd.concat([train[['user_profile', 'click_job_id']],test[['user_profile', 'click_job_id']]], axis=0)
data = data.fillna('-99')
data = data.replace({'':'-99'})
for i in ['user_profile', 'click_job_id']:
    sentence = []
    for line in list(data[i].values):
        sentence.append([str(l) for l in line.split(',')])
    print('training...{}'.format(i))
    model = Word2Vec(sentence, vector_size=8, window=2, min_count=1,epochs=10)
    outdf = []
    for line in tqdm(list(data[i].values)):
        sumarr = 0
        sl = line.split(',')
        for l in sl:
            sumarr = sumarr + model.wv[str(l)]
        outdf.append(sumarr/len(sl))
    tmp = pd.DataFrame(outdf)
    tmp.columns = ['{}'.format(i)+str(j) for j in tmp.columns]
    train_w2vDF = tmp[:train.shape[0]]
    test_w2vDF = tmp[train.shape[0]:].reset_index(drop=True)
    train_w2vDF.to_pickle('./data/train_w2vDF_{}.pkl'.format(i))
    test_w2vDF.to_pickle('./data/test_w2vDF_{}.pkl'.format(i))
train_w2vDF = pd.read_pickle('./data/train_w2vDF_user_profile.pkl')
test_w2vDF = pd.read_pickle('./data/test_w2vDF_user_profile.pkl')
train_w2vDF_click_job = pd.read_pickle('./data/train_w2vDF_click_job_id.pkl')
test_w2vDF_click_job = pd.read_pickle('./data/test_w2vDF_click_job_id.pkl')
#添加此特征后需在模型参数修改稠密特征维度，此处每个embedding为8维
train = pd.concat([train,train_w2vDF,train_w2vDF_click_job],axis=1)
test = pd.concat([test,test_w2vDF,test_w2vDF_click_job],axis=1)